In [648]:
import pandas as pd
import numpy as np

In [649]:
df = pd.read_csv('banknote.csv')

In [658]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df,test_size=0.2,random_state=42)
X_train = train_set.drop('class',axis=1)
y_train = train_set['class']
X_test = test_set.drop('class',axis=1)
y_test = test_set['class']

In [659]:
y_train_v = (y_train == 1)
y_test_v = (y_test == 1)

In [660]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=5,tol=None,random_state=42)
sgd_clf.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [661]:
predictions = sgd_clf.predict(X_test)
# (predictions == y_test_v)

In [654]:
y_train_pred = sgd_clf.predict(X_train)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_v, y_train_pred)

array([[606,   8],
       [  0, 483]])

This model is not sufficient to completely separate the two classes.

In [655]:
from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(sgd_clf, X_train, y_train_v, cv=5, method="decision_function")
y_scores

array([-420.57041343, -479.53146584,  192.78664805, ...,  207.45001952,
        321.13087588,  494.62779519])

In [656]:
sgd_clf.intercept_, sgd_clf.coef_

(array([ 129.52531349]),
 array([[-95.54213448, -64.04050432, -69.98052745,  -4.7058575 ]]))

We now use polynomial regression to improve our classification.

In [641]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly_features.fit_transform(X_train)

In [642]:
X_poly_test = poly_features.fit_transform(X_test)

In [643]:
poly_clf = SGDClassifier(max_iter=10,tol=None,random_state=42)
poly_clf.fit(X_poly_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=10, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [644]:
y_test_v = (y_test == 1)
y_test_pred = poly_clf.predict(X_poly_test)
confusion_matrix(y_test_v, y_test_pred)

array([[148,   0],
       [  0, 127]])

A polynomial model with degree 2 separates the two classes perfectly.
Let's try this out with Ridge regularization as well.

In [662]:
sgd_rdg = SGDClassifier(max_iter=8, tol=None, penalty="l2")
sgd_rdg.fit(X_poly_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=8, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [663]:
y_test_pred = sgd_rdg.predict(X_poly_test)
confusion_matrix(y_test_v, y_test_pred)

array([[148,   0],
       [  0, 127]])